1. 在DockerFile替换为以下代码，并Rebuild、重启
```shell
FROM gcr.io/deepnote-200602/templates/deepnote
RUN pip install bash_kernel && python -m bash_kernel.install
ENV DEFAULT_KERNEL_NAME "bash"
```

2. 在下方修改remote_port | 端口在：`4000-50000` 之间 | 建议改成难记的端口以免占用

3. 切勿修改/work/frp/frpc.ini | 相关守则参考 `https://www.ioiox.com/frp.html`

In [ ]:
frpV=$(wget -qO - https://api.github.com/repos/fatedier/frp/releases/latest | grep "tag_name" | cut -d"\"" -f4 |grep -oE "[^v]+") && wget -qO - https://github.com/fatedier/frp/releases/download/v${frpV}/frp_${frpV}_linux_amd64.tar.gz | tar -xzC /work && rm -rf /work/frp && mv /work/frp_${frpV}_linux_amd64 /work/frp
#-------------------------------------------------------
cat > /work/frp/frpc.ini << EOF
[common]
server_addr = frp2.ioiox.com
server_port = 7007
token = www.ioiox.com

[aria2tcp]
type = tcp
local_ip = 127.0.0.1
local_port = 6800
remote_port = 6800
#修改remote_port | 端口在：4000-50000 之间 | 建议改成难记的端口以免占用
EOF
#------------------------------------------------------
cat > /bin/pd <<\EOF
[[ -f /datasets/conf/passwd.conf ]] && [[ -z $(grep -oE "${1}" ) ]] && echo ${1} >> /datasets/conf/passwd.conf
EOF
chmod +rwx /bin/pd
rm -rf /datasets/*
wget --no-check-certificate -qO /datasets/DAria2.zip https://github.com/e9965/DAria2/blob/main/DAria2.zip?raw=true && unzip -qq /datasets/DAria2.zip -d /datasets/ && chmod +rwx /datasets/aria2.sh && chmod +rwx /datasets/sh.sh && rm -rf /datasets/DAria2.zip
cd /work/frp
./frpc -c ./frpc.ini & sudo bash /datasets/aria2.sh && stress-ng -c 2 -l 5 -t 180d > /dev/null 2>&1

--2020-12-26 03:31:14--  https://github.com/e9965/DAria2/blob/main/DAria2.zip?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/e9965/DAria2/raw/main/DAria2.zip [following]
--2020-12-26 03:31:14--  https://github.com/e9965/DAria2/raw/main/DAria2.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9965/DAria2/main/DAria2.zip [following]
--2020-12-26 03:31:14--  https://raw.githubusercontent.com/e9965/DAria2/main/DAria2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.200.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.200.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10715 (10K) [application/zip]
Saving to: ‘/datasets/DAria2.zip’

/datasets/DAria2